# Analysis Outline:
In this short project, Credit Card fraud dataset was analyzed and modelled using XGBoost and Model hyperparameters were optimized using hyperopt. 

In [1]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import xgboost

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve,auc
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score

from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

from scipy import interp

from itertools import cycle

from xgboost import XGBClassifier

%matplotlib inline

In [125]:
data = pd.read_csv("creditcard.csv")

In [57]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [58]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
Time      284807 non-null float64
V1        284807 non-null float64
V2        284807 non-null float64
V3        284807 non-null float64
V4        284807 non-null float64
V5        284807 non-null float64
V6        284807 non-null float64
V7        284807 non-null float64
V8        284807 non-null float64
V9        284807 non-null float64
V10       284807 non-null float64
V11       284807 non-null float64
V12       284807 non-null float64
V13       284807 non-null float64
V14       284807 non-null float64
V15       284807 non-null float64
V16       284807 non-null float64
V17       284807 non-null float64
V18       284807 non-null float64
V19       284807 non-null float64
V20       284807 non-null float64
V21       284807 non-null float64
V22       284807 non-null float64
V23       284807 non-null float64
V24       284807 non-null float64
V25       284807 non-null float64
V26  

In [59]:
data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.919560e-15,5.688174e-16,-8.769071e-15,2.782312e-15,-1.552563e-15,2.010663e-15,-1.694249e-15,-1.927028e-16,-3.137024e-15,...,1.537294e-16,7.959909e-16,5.367590e-16,4.458112e-15,1.453003e-15,1.699104e-15,-3.660161e-16,-1.206049e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [60]:
pandas_profiling.ProfileReport(data)

C:\Users\Taraneh\Anaconda3\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,31
Number of observations,284807
Total Missing (%),0.0%
Total size in memory,67.4 MiB
Average record size in memory,248.0 B
Numeric,30
Categorical,0
Boolean,1
Date,0
Text (Unique),0
Rejected,0


We have 30 variables including time, amount of transaction, and 28 unknown variables. Time is in seconds and the data shows transactions in 2 days. There are no non-values and all V1 to V28 variables have a mean of zero. Additionaly, the dataset is very imbalanced.

In [126]:
# Since it is an imbalanced data, we keep the default of having stratified split
X_train, X_test, y_train, y_test= train_test_split(data.drop('Class', axis=1), data.Class, 
                                                   test_size=0.3, random_state=123)

In [127]:
y_train.value_counts()

0    199032
1       332
Name: Class, dtype: int64

In [128]:
y_test.value_counts()

0    85283
1      160
Name: Class, dtype: int64

In [99]:
# defining the space for hyperparameter tuning

'''
hp.choice(label, options) — Returns one of the options, which should be a list or tuple.
hp.randint(label, upper) — Returns a random integer between the range [0, upper).
hp.uniform(label, low, high) — Returns a value uniformly between low and high.
hp.quniform(label, low, high, q) — Returns a value round(uniform(low, high) / q) * q, i.e it rounds the decimal values and returns an integer
hp.normal(label, mean, std) — Returns a real value that’s normally-distributed with mean and standard deviation sigma.
'''

space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180
    }


def hyperparameter_tuning(space):
    clf=xgboost.XGBClassifier(n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                         reg_alpha = int(space['reg_alpha']),min_child_weight=space['min_child_weight'],
                         colsample_bytree=space['colsample_bytree'])
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="rmse",
            early_stopping_rounds=10,verbose=False)

    pred = clf.predict(X_test)
    f1score = f1_score(y_test, pred>0.5)
    print ("SCORE:", f1score)
    #change the metric if you like
    return {'loss':-f1score, 'status': STATUS_OK }


# run the hyper paramter tuning
trials = Trials()
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

print (best)


hp.choice(label, options) — Returns one of the options, which should be a list or tuple.
hp.randint(label, upper) — Returns a random integer between the range [0, upper).
hp.uniform(label, low, high) — Returns a value uniformly between low and high.
hp.quniform(label, low, high, q) — Returns a value round(uniform(low, high) / q) * q, i.e it rounds the decimal values and returns an integer
hp.normal(label, mean, std) — Returns a real value that’s normally-distributed with mean and standard deviation sigma.

SCORE:                                                                                                                 
0.0                                                                                                                    
SCORE:                                                                                                                 
0.0                                                                                                                    
SCORE: 

0.7692307692307693                                                                                                     
SCORE:                                                                                                                 
0.7516483516483516                                                                                                     
SCORE:                                                                                                                 
0.756043956043956                                                                                                      
SCORE:                                                                                                                 
0.7645788336933044                                                                                                     
SCORE:                                                                                                                 
0.7608695652173914                      

In [100]:
# Model with best parameters 
clf = xgb.XGBClassifier(colsample_bytree= 0.9339241043628883,
 gamma = 1.390620403918958,
 max_depth=12,
 min_child_weight=8,
 reg_alpha = 4,
 reg_lambda = 0.6850663054088446, eta= 0.8, nthread=-1, n_estimators=50, seed=123)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[142127     20]
 [    50    207]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    142147
           1       0.91      0.81      0.86       257

    accuracy                           1.00    142404
   macro avg       0.96      0.90      0.93    142404
weighted avg       1.00      1.00      1.00    142404



## Adding scale_pos_weight hyperparameter since it is an imbalanced dataset:

In [120]:
# defining the space for hyperparameter tuning

'''
hp.choice(label, options) — Returns one of the options, which should be a list or tuple.
hp.randint(label, upper) — Returns a random integer between the range [0, upper).
hp.uniform(label, low, high) — Returns a value uniformly between low and high.
hp.quniform(label, low, high, q) — Returns a value round(uniform(low, high) / q) * q, i.e it rounds the decimal values and returns an integer
hp.normal(label, mean, std) — Returns a real value that’s normally-distributed with mean and standard deviation sigma.
'''

space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
       'scale_pos_weight': sum(y_train==0)/ sum(y_train==1)
    }




def hyperparameter_tuning(space):
    clf=xgboost.XGBClassifier(n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                         reg_alpha = int(space['reg_alpha']),min_child_weight=space['min_child_weight'],
                         colsample_bytree=space['colsample_bytree'], scale_pos_weight = space['scale_pos_weight'])
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="rmse",
            early_stopping_rounds=10,verbose=False)

    pred = clf.predict(X_test)
    f1score = f1_score(y_test, pred>0.5)
    print ("SCORE:", f1score)
    #change the metric if you like
    return {'loss':-f1score, 'status': STATUS_OK }


# run the hyper paramter tuning
trials = Trials()
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

print (best)


hp.choice(label, options) — Returns one of the options, which should be a list or tuple.
hp.randint(label, upper) — Returns a random integer between the range [0, upper).
hp.uniform(label, low, high) — Returns a value uniformly between low and high.
hp.quniform(label, low, high, q) — Returns a value round(uniform(low, high) / q) * q, i.e it rounds the decimal values and returns an integer
hp.normal(label, mean, std) — Returns a real value that’s normally-distributed with mean and standard deviation sigma.

SCORE:                                                                                                                 
0.6716867469879518                                                                                                     
SCORE:                                                                                                                 
0.7104000000000001                                                                                                     
SCORE: 

0.7521367521367521                                                                                                     
SCORE:                                                                                                                 
0.7692307692307693                                                                                                     
SCORE:                                                                                                                 
0.7308970099667774                                                                                                     
SCORE:                                                                                                                 
0.7773851590106007                                                                                                     
SCORE:                                                                                                                 
0.763888888888889                       

In [121]:
# Model with best parameters 
clf = xgb.XGBClassifier(colsample_bytree= 0.888725180329427,
 gamma = 3.35094873745297,
 max_depth=16,
 min_child_weight=0,
 reg_alpha = 4,
 reg_lambda = 0.43787042553278144,scale_pos_weight= sum(y_train==0)/ sum(y_train==1), eta= 0.3, nthread=-1, n_estimators=180, seed=123)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[142108     39]
 [    41    216]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    142147
           1       0.85      0.84      0.84       257

    accuracy                           1.00    142404
   macro avg       0.92      0.92      0.92    142404
weighted avg       1.00      1.00      1.00    142404



Adding scale_pos_weight hyperparameter has improved recall of fraud cases (minority class) which is important in fraud detection to minimize False Negative instances.